In [ ]:
import httpx
import pytest
from unittest.mock import AsyncMock, patch, Mock
from fh_saas.utils_api import AsyncAPIClient, bearer_token_auth, api_key_auth

## Test AsyncAPIClient

In [ ]:
@pytest.mark.asyncio
async def test_successful_get_request():
    """Test successful GET request returns JSON data"""
    
    # Mock response
    mock_response = Mock(spec=httpx.Response)
    mock_response.status_code = 200
    mock_response.json.return_value = {'users': [{'id': 1, 'name': 'Alice'}]}
    mock_response.raise_for_status = Mock()
    
    # Mock client
    with patch('httpx.AsyncClient') as MockClient:
        mock_client_instance = AsyncMock()
        mock_client_instance.request.return_value = mock_response
        MockClient.return_value = mock_client_instance
        
        # Test
        async with AsyncAPIClient('https://api.example.com', timeout=10) as client:
            data = await client.get_json('/users')
            
            assert data == {'users': [{'id': 1, 'name': 'Alice'}]}
            mock_client_instance.request.assert_called_once()

# Run test
await test_successful_get_request()
print("✅ Test passed: Successful GET request")

✅ Test passed: Successful GET request


In [ ]:
@pytest.mark.asyncio
async def test_retry_on_500_error():
    """Test retry logic activates on 500 errors"""
    
    # Mock 500 error response
    error_response = Mock(spec=httpx.Response)
    error_response.status_code = 500
    error_response.raise_for_status.side_effect = httpx.HTTPStatusError(
        "Server Error", request=Mock(), response=error_response
    )
    
    with patch('httpx.AsyncClient') as MockClient:
        mock_client_instance = AsyncMock()
        mock_client_instance.request.return_value = error_response
        MockClient.return_value = mock_client_instance
        
        # Test - should retry 3 times then raise
        async with AsyncAPIClient('https://api.example.com') as client:
            try:
                await client.request('GET', '/users')
                assert False, "Should have raised HTTPStatusError"
            except httpx.HTTPStatusError:
                # Verify 3 retry attempts
                assert mock_client_instance.request.call_count == 3

# Run test
await test_retry_on_500_error()
print("✅ Test passed: Retry on 500 error")

✅ Test passed: Retry on 500 error


In [ ]:
@pytest.mark.asyncio
async def test_retry_on_429_rate_limit():
    """Test retry logic activates on 429 rate limit"""
    
    # Mock 429 error response
    error_response = Mock(spec=httpx.Response)
    error_response.status_code = 429
    error_response.raise_for_status.side_effect = httpx.HTTPStatusError(
        "Rate Limited", request=Mock(), response=error_response
    )
    
    with patch('httpx.AsyncClient') as MockClient:
        mock_client_instance = AsyncMock()
        mock_client_instance.request.return_value = error_response
        MockClient.return_value = mock_client_instance
        
        # Test - should retry 3 times then raise
        async with AsyncAPIClient('https://api.example.com') as client:
            try:
                await client.request('GET', '/users')
                assert False, "Should have raised HTTPStatusError"
            except httpx.HTTPStatusError:
                # Verify 3 retry attempts
                assert mock_client_instance.request.call_count == 3

# Run test
await test_retry_on_429_rate_limit()
print("✅ Test passed: Retry on 429 rate limit")

✅ Test passed: Retry on 429 rate limit


In [ ]:
@pytest.mark.asyncio
async def test_no_retry_on_400_error():
    """Test that 400-level errors do NOT retry"""
    
    # Mock 404 error response
    error_response = Mock(spec=httpx.Response)
    error_response.status_code = 404
    error_response.raise_for_status.side_effect = httpx.HTTPStatusError(
        "Not Found", request=Mock(), response=error_response
    )
    
    with patch('httpx.AsyncClient') as MockClient:
        mock_client_instance = AsyncMock()
        mock_client_instance.request.return_value = error_response
        MockClient.return_value = mock_client_instance
        
        # Test - should NOT retry, fail immediately
        async with AsyncAPIClient('https://api.example.com') as client:
            try:
                await client.request('GET', '/users')
                assert False, "Should have raised HTTPStatusError"
            except httpx.HTTPStatusError:
                # Verify only 1 attempt (no retries)
                assert mock_client_instance.request.call_count == 1

# Run test
await test_no_retry_on_400_error()
print("✅ Test passed: No retry on 400-level errors")

✅ Test passed: No retry on 400-level errors


## Test Auth Helpers

In [ ]:
def test_bearer_token_auth():
    """Test Bearer token header generation"""
    headers = bearer_token_auth('my_secret_token')
    assert headers == {'Authorization': 'Bearer my_secret_token'}
    print("✅ Test passed: Bearer token auth")

test_bearer_token_auth()

✅ Test passed: Bearer token auth


In [ ]:
def test_api_key_auth_default():
    """Test API key with default header name"""
    headers = api_key_auth('my_api_key')
    assert headers == {'X-API-Key': 'my_api_key'}
    print("✅ Test passed: API key auth (default)")

test_api_key_auth_default()

✅ Test passed: API key auth (default)


In [ ]:
def test_api_key_auth_custom():
    """Test API key with custom header name"""
    headers = api_key_auth('my_api_key', 'X-Custom-Key')
    assert headers == {'X-Custom-Key': 'my_api_key'}
    print("✅ Test passed: API key auth (custom)")

test_api_key_auth_custom()

✅ Test passed: API key auth (custom)
